In [3]:
#只需要添加nn.BatchNorm层并指定对二维卷积的通道(axis=1)进行批量归一化。

from mxnet.gluon import nn
net=nn.Sequential()
with net.name_scope():
    #first convolutional layer
    net.add(nn.Conv2D(channels=20, kernel_size=5))
    net.add(nn.BatchNorm(axis=1))
    net.add(nn.Activation('relu'))
    net.add(nn.MaxPool2D(pool_size=2,strides=3))
    #second convolutional layer
    net.add(nn.Conv2D(channels=50,kernel_size=3))
    net.add(nn.BatchNorm(axis=1))
    net.add(nn.Activation('relu'))
    net.add(nn.MaxPool2D(pool_size=2,strides=2))
    net.add(nn.Flatten())
    #the first connection layer
    net.add(nn.Dense(128,activation="relu"))
    #second connection layer
    net.add(nn.Dense(10))

In [7]:
import sys
sys.path.append('..')
import utils
from mxnet import autograd
from mxnet import gluon
from mxnet import nd

ctx=utils.try_gpu()
net.initialize(ctx=ctx)

batch_size=256
train_data,test_data=utils.load_data_fashion_mnist(batch_size)

softmax_cross_entropy=gluon.loss.SoftmaxCrossEntropyLoss()
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.2})

for epoch in range(3):
    train_loss=0.
    train_acc=0.
    for data, label in train_data:
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data.as_in_context(ctx))
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)
            

        train_loss+=nd.mean(loss).asscalar()
        train_acc+=utils.accuracy(output,label)
    test_acc=utils.evaluate_accuracy(test_data,net,ctx)
    
    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
        epoch, train_loss/len(train_data),
        train_acc/len(train_data), test_acc))
            

/home/wuying/miniconda3/lib/python3.6/site-packages/mxnet/gluon/parameter.py:320: UserWarning: Parameter sequential2_conv0_weight is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "Set force_reinit=True to re-initialize."%self.name)
/home/wuying/miniconda3/lib/python3.6/site-packages/mxnet/gluon/parameter.py:320: UserWarning: Parameter sequential2_conv0_bias is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "Set force_reinit=True to re-initialize."%self.name)
/home/wuying/miniconda3/lib/python3.6/site-packages/mxnet/gluon/parameter.py:320: UserWarning: Parameter sequential2_batchnorm0_gamma is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "Set force_reinit=True to re-initialize."%self.name)
/home/wuying/miniconda3/lib/python3.6/site-packages/mxnet/gluon/parameter.py:320: UserWarning: Parameter sequential2_batchnorm0_beta is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "Set force_r

Epoch 0. Loss: 0.589865, Train acc 0.783337, Test acc 0.821915
Epoch 1. Loss: 0.373172, Train acc 0.862730, Test acc 0.848257
Epoch 2. Loss: 0.323295, Train acc 0.880826, Test acc 0.872696
